In [ ]:
import sys
sys.path.append('../libs/')

import os

In [ ]:
def make_visual_conf(main_index, role_index, colors = {'red', 'blue', '#f0a105'}):
    basic_conf = \
u'''
[labels]

{}

[drawing]

Arg fgColor:black, bgColor:#FF7070, borderColor:darken
Pred fgColor:black, bgColor:lightgreen, borderColor:darken

{}

'''
    full_index = {}
    for e in role_index.iteritems():
        full_index.update(e[1])
    full_index.update(main_index)

    roles = unicode()
    for role_str, role_real_name in full_index.iteritems():
        roles += u'{} | {}\n'.format(role_str, role_real_name)
    
    bad_roles_str = u''
    for (_, role_indexes), color in zip(role_index.iteritems(), colors):
        for role, _ in role_indexes.iteritems():
            bad_roles_str += u'{}\tcolor:{}, dashArray:-, arrowHead:triangle-5\n'.format(role, color)
    
    return basic_conf.format(roles, bad_roles_str).encode('utf8') 


def make_annotations_conf(role_index):
    basic_conf = \
u'''
[entities]

Arg
Pred

[attributes]

[relations]

{}


[events]

'''

    roles = str()
    for role_name, _ in sorted(role_index.iteritems(), key = lambda e: e[0]):
        roles += u'{} Arg1:Pred, Arg2:Arg\n'.format(role_name)
    
    return basic_conf.format(roles).encode('utf8')

In [ ]:
import shutil
import json
from pylingtools.format_brat import load_brat_annotation_dir, save_list_of_brat_objects
from pylingtools.format_brat import BratRealtion


def load_brat_annotation_dir_recur(input_dir_path):
    result = {}
    for file_dir in os.listdir(input_dir_path):
        true_path = os.path.join(input_dir_path, file_dir)
        if not os.path.isdir(true_path):
            continue
        
        data = load_brat_annotation_dir(true_path)
        result.update({file_dir + '/' + d :v for d, v in data.iteritems()})
    
    return result


def load_role_index(path):
    with open(path, 'r') as f:
        role_index = json.load(f)
    
    reversed_index = {'Role_' + str(v): k for k, v in role_index.iteritems()} 
    return reversed_index

        
def process_part(num, part_path, main_ann):
    path_ann = load_brat_annotation_dir_recur(part_path)
    role_index = load_role_index(os.path.join(part_path, 'rel_name_index.json'))
    sub_annots = {}
    new_role_index = {}
    for int_path, vals in path_ann.iteritems():
        offset_index = main_ann[int_path]['offset_index']
        ents, rels = vals['entities'], vals['relations']
        loc_offset_index = {}
        for ent in ents:
            loc_offset_index[ent.bid] = (ent.begin, ent.end)

        annots = list()
        for rel in rels:
            if not role_index[rel.tp].startswith('false'):
                continue

            old_name = role_index[rel.tp]
            new_rel = rel
            new_rel.dep = offset_index[loc_offset_index[rel.dep]]
            new_rel.head = offset_index[loc_offset_index[rel.head]]
            new_rel.tp = 'false_' + str(num) + '_' + new_rel.tp
            #print rel.tp
            new_role_index[new_rel.tp] = old_name
            annots.append(new_rel)

        sub_annots[int_path] = annots
    
    return sub_annots, new_role_index


def process_main_rels(main_path):
    main_ann = load_brat_annotation_dir_recur(main_path)
    
    reversed_index = load_role_index(os.path.join(main_path, 'rel_name_index.json'))
        
    new_main_ann = {}
    for mpath, anns in main_ann.iteritems():
        main_ents, main_rels = anns['entities'], anns['relations']
        
        offset_index = {}
        for ent in main_ents:
            offset_index[(ent.begin, ent.end)] = ent.bid
    
        new_main_rels = list()
        for rel in main_rels:
            if reversed_index[rel.tp].startswith('false'):
                continue

            new_main_rels.append(rel)
    
        main_rels = new_main_rels
        new_main_ann[mpath] = {'entities' : main_ents, 
                               'relations' : main_rels,
                              'offset_index' : offset_index}
    
    return new_main_ann, reversed_index
        

def merge_annotations(dir_path_list, output_dir_path):
    main_path = dir_path_list[0]
    shutil.copytree(main_path, output_dir_path)
    
    main_ann, main_role_index = process_main_rels(main_path)
    
    sub_annots = {}
    for num, path in enumerate(dir_path_list):
        ann = process_part(num, path, main_ann)
        sub_annots[path] = ann
    
    result = {}
    role_index = {path : e[1] for path, e in sub_annots.iteritems()}
    annot_index = {path : e[0] for path, e in sub_annots.iteritems()}
    for file_path, main_annots in main_ann.iteritems():
        main_entities, main_rels = main_annots['entities'], main_annots['relations']
        all_annots = main_rels
        
        rel_id = max(int(rel.bid) for rel in main_rels)
        
        for path, annots in annot_index.iteritems():
            rel_annots = annots[file_path]
            
            for rel in rel_annots:
                rel_id += 1
                rel.bid = rel_id
                all_annots.append(rel)
            
        result[file_path] = (main_entities, all_annots)
    
    
        
    for file_path, annots in result.iteritems():
        output_file_path = os.path.join(output_dir_path, file_path + '.ann')
        save_list_of_brat_objects(annots[0] + annots[1], output_file_path)
    
    with open(os.path.join(output_dir_path, 'visual.conf'), 'w') as f:
        f.write(make_visual_conf(main_role_index, role_index))
    
    full_index = {}
    for e in role_index.iteritems():
        full_index.update(e[1])
    
    full_index.update(main_role_index)
    
    with open(os.path.join(output_dir_path, 'annotation.conf'), 'w') as f:
        f.write(make_annotations_conf(full_index))
    

In [ ]:
!export PYTHONPATH="$PYTHONPATH:../libs/" && python ../libs/framebank_preprocessing/convert_corpus_to_brat.py \
--inputFile=../data/test_data_annotated_with_no_pred_lemma.json \
--outputDir=../data/brat/test_data_annotated_with_no_pred_lemma/ \
--converter=srl

In [ ]:
!export PYTHONPATH="$PYTHONPATH:../libs/" && python ../libs/framebank_preprocessing/convert_corpus_to_brat.py \
--inputFile=../data/test_data_annotated_1.json \
--outputDir=../data/brat/test_data_annotated_srl_1/ \
--converter=srl

In [ ]:
!export PYTHONPATH="$PYTHONPATH:../libs/" && python ../libs/framebank_preprocessing/convert_corpus_to_brat.py \
--inputFile=../data/test_data_annotated_1.json \
--outputDir=../data/brat/test_data_annotated_syn_1 \
--converter=syn

In [ ]:
input_dir_path = ["../data/brat/test_data_annotated_with_simple", 
                  "../data/brat/test_data_annotated_with_complex",
                  "../data/brat/test_data_annotated_with_no_pred_lemma"]
output_dir_path = "../data/brat_result/"
merge_annotations(input_dir_path, output_dir_path)